
 <a id="1"></a>
# <p style="background-color:#627D78;font-family:newtimeroman;color:#D5CABD;font-size:150%;text-align:center;border-radius:20px 60px;">Importing Libraries</p>

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import string
# for Tokenization
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
# for removing accented and special chracters
import unicodedata
# for stopwords Removal
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
# for calculating Polarity and Subjectivity
from textblob import TextBlob
import plotly.express as px
#nlp = spacy.load("en_core_web_sm")
# function for making ngrams
from nltk.util import ngrams 
import collections
# for Wordscloud
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import zero_one_loss

In [ ]:
dataset='../input/intent-classification-enfr/intent_classification.csv'
df=pd.read_csv(dataset)

### lets check the head of the dataset

In [ ]:
df.head(10)

In [ ]:
df.shape

 ### lets check if the dataset has any Missing Values

In [ ]:
df.isnull()

 #### lets check the Descriptive Summary of the Dataset

In [ ]:
df.describe()

#### lets check the summary of Date



In [ ]:
df.describe(include = 'object')

#### lets check the Value Counts for Variation 

In [ ]:
df['text-en'].value_counts()

### Lets calculate the length of text-en

In [ ]:
# Lets calculate the length of the Reviews
df['length'] = df['text-en'].apply(len)

### Text Polarity

It is the expression that determines the sentimental aspect of an opinion. In textual data, the result of sentiment analysis can be determined for each entity in the sentence, document or sentence. The sentiment polarity can be determined as positive, negative and neutral.


In [ ]:
# Lets calculate the Polarity of the Reviews
def get_polarity(text):
    textblob = TextBlob(str(text.encode('utf-8')))
    pol = textblob.sentiment.polarity
    return pol

# lets apply the function
df['polarity'] = df['text-en'].apply(get_polarity)

### Text Subjectivity

In natural language, subjectivity refers to expression of opinions, evaluations, feelings, and speculations and thus incorporates sentiment. Subjective text is further classified with sentiment or polarity.

In [ ]:
def get_subjectivity(text):
    textblob = TextBlob(str(text.encode('utf-8')))
    subj = textblob.sentiment.subjectivity
    return subj

# lets apply the Function
df['subjectivity'] = df['text-en'].apply(get_subjectivity)

In [ ]:
## lets summarize the Newly Created Features
df[['length','polarity','text-en']].describe()

In [ ]:
## lets summarize the Newly Created Features
df[['length','polarity','subjectivity']].describe()

In [ ]:
# lets create a Part of speech Dictionary
pos_dic = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def pos_check(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt

### Nouns

A noun is a word that functions as the name of a specific object or set of objects, such as living creatures, places, actions, qualities, states of existence, or ideas. However, noun is not a semantic category, so that it cannot be characterized in terms of its meaning.

In [ ]:
# lets calculate the count of Nouns in the Text
df['noun_count'] = df['text-en'].apply(lambda x: pos_check(x, 'noun'))

### Verbs

A verb, from the Latin verbum meaning word, is a word that in syntax conveys an action, an occurrence, or a state of being. In the usual description of English, the basic form, with or without the particle to, is the infinitive. In many languages, verbs are inflected to encode tense, aspect, mood, and voice.

In [ ]:
df['verb_count'] = df['text-en'].apply(lambda x: pos_check(x, 'verb'))

In [ ]:
df[['noun_count','verb_count']].describe()

### **Here calculate the char_count & word_count & word_density & punctuation_count**

In [ ]:
# calculating the Character Count in the Reviews
df['char_count'] = df['text-en'].apply(len)

In [ ]:
# calculating the Word Count
df['word_count'] = df['text-en'].apply(lambda x: len(x.split()))

In [ ]:
# Calculating the Word Density
df['word_density'] = df['char_count'] / (df['word_count']+1)

In [ ]:
punctuation = string.punctuation

# Calculating the Punctuation Count
df['punctuation_count'] = df['text-en'].apply(lambda x: len("".join(_ for _ in x if _ in punctuation))) 

In [ ]:
## lets summarize the Newly Created Features
df[['char_count','word_count','word_density','punctuation_count']].describe()

### Adjectives

In linguistics, an adjective is a word that modifies a noun or noun phrase or describes its referent. Its semantic role is to change information given by the noun. Adjectives are one of the main parts of speech of the English language, although historically they were classed together with nouns.

In [ ]:
df['adj_count'] = df['text-en'].apply(lambda x: pos_check(x, 'adj'))

### Adverbs

An adverb is a word or an expression that modifies a verb, adjective, another adverb, determiner, clause, preposition, or sentence. Adverbs typically express manner, place, time, frequency, degree, level of certainty, etc., answering questions such as how?, in what way?, when?, where?, and to what extent?

In [ ]:
df['adv_count'] = df['text-en'].apply(lambda x: pos_check(x, 'adv'))

### Pronouns

A pronoun (I, me, he, she, herself, you, it, that, they, each, few, many, who, whoever, whose, someone, everybody, etc.) is a word that takes the place of a noun. In the sentence Joe saw Jill, and he waved at her, the pronouns he and her take the place of Joe and Jill, respectively.

In [ ]:
# lets calculate the count of Pronouns in the Text
df['pron_count'] = df['text-en'].apply(lambda x: pos_check(x, 'pron'))

In [ ]:
## lets summarize the Newly Created features
df[['adj_count','adv_count','pron_count']].describe()

### Now we can Clear the data

In [ ]:
# First lets remove Punctuations from the Reviews
def punctuation_removal(messy_str):
    clean_list = [char for char in messy_str if char not in string.punctuation]
    clean_str = ''.join(clean_list)
    return clean_str

df['text-en'] = df['text-en'].apply(punctuation_removal)

In [ ]:
def drop_numbers(list_text):
    list_text_new = []
    for i in list_text:
        if not re.search('\d', i):
            list_text_new.append(i)
    return ''.join(list_text_new)

df['text-en'] = df['text-en'].apply(drop_numbers)

In [ ]:
df['text-en'].head(10).sum()

In [ ]:
sentence_data='''can you walk me through setting up direct deposits
to my bank of internet savings accounti want to switch to 
direct depositset up direct deposit
for mehow do i go about setting up direct depositi need 
to get my paycheck direct deposited to my chase accountwhat are the steps to 
set up direct deposit to my chase accountif i would like to set up direct 
deposit how do i do ithow do i direct deposit my checkwhat do i need to set u
p direct depositid like to have my paycheck direct 
deposited to my chase account'''
nltk_tokens = nltk.sent_tokenize(sentence_data)
print (nltk_tokens)

### Tokenization

In Python tokenization basically refers to splitting up a larger body of text into smaller lines, words or even creating words for a non-English language. The various tokenization functions in-built into the nltk module

In [ ]:
# Words Tokenization
nltk_tokens = nltk.word_tokenize(sentence_data)
print (nltk_tokens)

In [ ]:
# Create a function to remove special characters
def remove_special_characters(text):
    pat = r'[^a-zA-z0-9]' 
    return re.sub(pat, ' ', text)
 
# lets apply this function
df['text-en'] = df.apply(lambda x: remove_special_characters(x['text-en']), axis = 1)

In [ ]:
df['text-en'].head(10).sum()

### Removing Accented Characters

Accented characters are important elements which are used to signify emphasis on a particular word during pronunciation or understanding. In some instances, the accent mark also clarifies the meaning of a word, which might be different without the accent. While their use in English is largely limited but there are very good chances that you will come across accented characters/letters in a free text corpus. 


### Stopwords

Stop words are a set of commonly used words in any language. For example, in English, “the”, “is” and “and”, would easily qualify as stop words. In NLP and text mining applications, stop words are used to eliminate unimportant words, allowing applications to focus on the important words instead.

In [ ]:
# Now lets Remove the Stopwords

# targeting only English Stopwords
stop = stopwords.words('english')
stop_words = []
from nltk.tokenize import word_tokenize

text = '''can you walk me through setting up direct deposits to my bank of
internet savings accounti want to switch to direct depositset up direct deposit
for mehow do i go about setting up direct depositi need to get my paycheck direct 
deposited to my chase accountwhat are the steps to set up direct deposit to my chase 
accountif i would like to set up direct deposit how do i do ithow do i direct deposit
my checkwhat do i need to set up direct depositid like to have 
my paycheck direct deposited to my chase account'''
text_tokens = word_tokenize(text)

tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]

print(tokens_without_sw)


### Lemmatization

Though stemming and lemmatization both generate the root form of inflected/desired words, but lemmatization is an advanced form of stemming. Stemming might not result in actual word, whereas lemmatization does conversion properly with the use of vocabulary, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma.
Before using lemmatization, we should be aware that it is considerably slower than stemming, so performance should be kept in mind before choosing stemming or lemmatization.

In [ ]:
# for lemmatization
import spacy
nlp = spacy.load('en_core_web_sm',parse=True,tag=True, entity=True)

In [ ]:
# function for stemming
def get_stem(text):
    stemmer = nltk.porter.PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

# call function
get_stem("can you walk me through setting up direct deposits to my bank of internet savings accounti want to switch to direct depositset up direct deposit for mehow do i go about setting up direct depositi need to get my paycheck direct deposited to my chase accountwhat are the steps to set up direct deposit to my chase accountif i would like to set up direct deposit how do i do ithow do i direct deposit my checkwhat do i need to set up direct depositid like to have my paycheck direct deposited to my chase account ")

In [ ]:
# function to remove special characters
def get_lem(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

# call function
get_lem("can you walk me through set up direct deposit to my bank of internet save accounti want to switch to direct depositset up direct deposit for mehow do i go about set up direct depos need to get my paycheck direct deposit to my chase accountwhat are the step to set up direct deposit to my chase accountif i would like to set up direct deposit how do i do ithow do i direct deposit my checkwhat do i need to set up direct depositid like to have my paycheck direct deposit to my chase account ")

### Bag of Words

The bag-of-words model is a simplifying representation used in natural language processing and information retrieval (IR). In this model, a text (such as a sentence or a document) is represented as the bag (multiset) of its words, disregarding grammar and even word order but keeping multiplicity.

In [ ]:
## Cleaning the Data

corpus = []

for i in range(0, 3150):
    review = re.sub('[^a-zA-Z]', ' ', df['text-en'][i])  ## Removing all Unecessary items
    review = review.lower()                                         ## Converting into Lower Case
    review = review.split()
    ps = PorterStemmer()                                            ## Stemming
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]  ## Removing Stopwords
    review = ' '.join(review)
    corpus.append(review)

### Text Polarity

It is the expression that determines the sentimental aspect of an opinion. In textual data, the result of sentiment analysis can be determined for each entity in the sentence, document or sentence. The sentiment polarity can be determined as positive, negative and neutral.

In [ ]:
# Lets calculate the Polarity of the Reviews
def get_polarity(text):
    textblob = TextBlob(str(text.encode('utf-8')))
    pol = textblob.sentiment.polarity
    return pol

# lets apply the function
df['polarity'] = df['text-en'].apply(get_polarity)

### Text Subjectivity

In natural language, subjectivity refers to expression of opinions, evaluations, feelings, and speculations and thus incorporates sentiment. Subjective text is further classified with sentiment or polarity.

In [ ]:
 #Lets calculate the Subjectvity of the Reviews
def get_subjectivity(text):
    textblob = TextBlob(str(text.encode('utf-8')))
    subj = textblob.sentiment.subjectivity
    return subj

# lets apply the Function
df['subjectivity'] = df['text-en'].apply(get_subjectivity)

In [ ]:
## Visualizing Polarity and Subjectivity

plt.rcParams['figure.figsize'] = (10, 4)

plt.subplot(1, 2, 1)
sns.distplot(df['polarity'])

plt.subplot(1, 2, 2)
sns.distplot(df['subjectivity'])

plt.suptitle('Distribution of Polarity and Subjectivity')
plt.show()

In [ ]:
# lets check relation between Polarity and Subjectivity

sns.scatterplot(df['polarity'], df['subjectivity'])
plt.title('Polarity vs Subjectivity')
plt.show()

In [ ]:
sns.lineplot(data=df.drop(['subjectivity'], axis=1))

In [ ]:
sns.lineplot(data=df.drop(['polarity'], axis=1))

In [ ]:
plt.hist(df["polarity"], bins = 30)

In [ ]:
plt.hist(df["subjectivity"], bins = 30)

In [ ]:
plt.hist(df['subjectivity'], color = 'b')

## Visualizing the Most Frequent Words

## lets plot the Wordscloud

In [ ]:

cv = CountVectorizer(stop_words = 'english')
words = cv.fit_transform(df['text-en'])
sum_words = words.sum(axis=0)

words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)

wordcloud = WordCloud(background_color = 'lightcyan', width = 2000, height = 2000).generate_from_frequencies(dict(words_freq))

plt.style.use('fivethirtyeight')
plt.figure(figsize=(10, 10))
plt.axis('off')
plt.imshow(wordcloud)
plt.title("Vocabulary from Reviews", fontsize = 20)
plt.show()

### Make feature_extraction

In [ ]:
# creating bag of words
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
cv = CountVectorizer(max_features=2)
df=cv.fit_transform(corpus)
#X = cv.fit_transform(corpus).toarray()
#X=df['']
#y = df.iloc[:, -1].values
X = df[:,:-1].toarray()
y = df[:,:-1].toarray()

print(X.shape)
print(y.shape)

In [ ]:
X[:10]

In [ ]:
y[:10]

 <a id="1"></a>
 <p style="background-color:#627D78;font-family:newtimeroman;color:#D5CABD;font-size:150%;text-align:center;border-radius:20px 60px;">we can split the data and using LogisticRegrission</p>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train

In [ ]:
LogisticRegressionModel = LogisticRegression(penalty='l2',solver='sag',C=1.0,random_state=33)
LogisticRegressionModel.fit(X_train, y_train)


In [ ]:
#Calculating Details
print('LogisticRegressionModel Train Score is : ' , LogisticRegressionModel.score(X_train, y_train))
print('LogisticRegressionModel Test Score is : ' , LogisticRegressionModel.score(X_test, y_test))
print('LogisticRegressionModel Classes are : ' , LogisticRegressionModel.classes_)
print('LogisticRegressionModel No. of iteratios is : ' , LogisticRegressionModel.n_iter_)

In [ ]:
#Calculating Prediction
y_pred = LogisticRegressionModel.predict(X_test)
y_pred_prob = LogisticRegressionModel.predict_proba(X_test)
print('Predicted Value for LogisticRegressionModel is : ' , y_pred[:10])
print('Prediction Probabilities Value for LogisticRegressionModel is : ' , y_pred_prob[:10])


In [ ]:
#Calculating Confusion Matrix
CM = confusion_matrix(y_test, y_pred)
print('Confusion Matrix is : \n', CM)


In [ ]:
# drawing confusion matrix
sns.heatmap(CM)
plt.show()


In [ ]:
#Calculating Accuracy Score  : ((TP + TN) / float(TP + TN + FP + FN))
AccScore = accuracy_score(y_test, y_pred, normalize=True)
print('Accuracy Score is : ', AccScore)


 <a id="1"></a>
# <p style="background-color:#627D78;font-family:newtimeroman;color:#D5CABD;font-size:150%;text-align:center;border-radius:20px 60px;">In the end, we see that we got the best score</p>